In [ ]:
#!pip install torch torchvision pillow numpy matplotlib

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

### Colab 마운트

In [67]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'test'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change directory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/test


필요한 라이브러리 import

In [58]:
import os
import shutil
import itertools
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

### 만약 파일을 실행시켜보고 싶다면 아래 코드를 실행시켜야 함
- directory에 유의 -> dataset4에 대한 경로를 집어넣어야 함
- 아래 코드는 각 파일 안에 사진 명을 older_face1, older_face2, ..., younger_face1, younger_face2,... 이런 형식으로 변환해주는 코드

In [ ]:
import os

# Define the directory containing the images
directory = '/content/drive/MyDrive/deep_learning/dataset4/older'

# List all files in the directory
files = os.listdir(directory)

# Filter for image files (jpg and png)
image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]

# Sort the files to maintain order
image_files.sort()

# Loop through and rename each file
for idx, filename in enumerate(image_files):
    # Construct the new file name
    new_name = f'older_face{idx+1}{os.path.splitext(filename)[1]}'  # Preserves the original file extension

    # Full paths for the source and destination files
    src = os.path.join(directory, filename)
    dst = os.path.join(directory, new_name)

    # Rename the file
    os.rename(src, dst)

    print(f'Renamed {src} to {dst}')

In [ ]:
import os

# Define the directory containing the images
directory = '/content/drive/MyDrive/deep_learning/dataset4/younger'

# List all files in the directory
files = os.listdir(directory)

# Filter for image files (jpg and png)
image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]

# Sort the files to maintain order
image_files.sort()

# Loop through and rename each file
for idx, filename in enumerate(image_files):
    # Construct the new file name
    new_name = f'younger_face{idx+1}{os.path.splitext(filename)[1]}'  # Preserves the original file extension

    # Full paths for the source and destination files
    src = os.path.join(directory, filename)
    dst = os.path.join(directory, new_name)

    # Rename the file
    os.rename(src, dst)

    print(f'Renamed {src} to {dst}')

### 사진 파일 불러오기

'dataset' 파일 안에 'older' 파일과 'younger' 파일이 있는 형태여야 함

In [ ]:
# Define the base directories
target_dir = '/content/drive/MyDrive/deep_learning/dataset4'
older_dir = os.path.join(target_dir, 'older')
younger_dir = os.path.join(target_dir, 'younger')

### transformations, dataloaders, batch size

In [ ]:
# Step 3: Run the PyTorch code
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    #transforms.RandomRotation(degrees=10),
    #transforms.RandomHorizontalFlip(),
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define dataset and dataloaders
older_dataset = datasets.ImageFolder(root=older_dir, transform=transform)
younger_dataset = datasets.ImageFolder(root=younger_dir, transform=transform)

# Ensure drop_last=True to handle incomplete batches
batch_size = 32
older_dataloader = DataLoader(older_dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
younger_dataloader = DataLoader(younger_dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)

# Example: Print first few images to verify
older_img, older_label = next(iter(older_dataloader))
younger_img, younger_label = next(iter(younger_dataloader))

print("Shape of older image batch:", older_img.shape)
print("Shape of younger image batch:", younger_img.shape)

Shape of older image batch: torch.Size([32, 3, 256, 256])
Shape of younger image batch: torch.Size([32, 3, 256, 256])


### generator, discriminator

In [ ]:
import torch.nn as nn

# Define the Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)


### cycleGAN 정의하기

In [ ]:
class CycleGAN(nn.Module):
    def __init__(self):
        super(CycleGAN, self).__init__()
        self.G_XtoY = Generator()
        self.G_YtoX = Generator()
        self.D_X = Discriminator()
        self.D_Y = Discriminator()

    def forward(self):
        pass

# Instantiate the model and move it to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cycle_gan = CycleGAN().to(device)
print(device)


cuda


### loss function, optimizer

In [ ]:
import itertools
import torch.optim as optim

# Loss functions
criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()

# Optimizers
optimizer_G = optim.Adam(itertools.chain(cycle_gan.G_XtoY.parameters(), cycle_gan.G_YtoX.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_X = optim.Adam(cycle_gan.D_X.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D_Y = optim.Adam(cycle_gan.D_Y.parameters(), lr=0.0002, betas=(0.5, 0.999))


### model training

In [ ]:
# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    for i, (data_X, data_Y) in enumerate(zip(older_dataloader, younger_dataloader)):
        # Get real images
        real_X = data_X[0].to(device)
        real_Y = data_Y[0].to(device)

        # Adversarial ground truths with the same shape as discriminator outputs
        valid = torch.ones(real_X.size(0), 1, 15, 15, dtype=torch.float, requires_grad=False).to(device)
        fake = torch.zeros(real_X.size(0), 1, 15, 15, dtype=torch.float, requires_grad=False).to(device)

        # ----------------------
        #  Train Generators
        # ----------------------

        optimizer_G.zero_grad()

        # Identity loss
        loss_id_X = criterion_identity(cycle_gan.G_YtoX(real_X), real_X)
        loss_id_Y = criterion_identity(cycle_gan.G_XtoY(real_Y), real_Y)

        loss_identity = (loss_id_X + loss_id_Y) / 2

        # GAN loss
        fake_Y = cycle_gan.G_XtoY(real_X)
        loss_GAN_XtoY = criterion_GAN(cycle_gan.D_Y(fake_Y), valid)

        fake_X = cycle_gan.G_YtoX(real_Y)
        loss_GAN_YtoX = criterion_GAN(cycle_gan.D_X(fake_X), valid)

        loss_GAN = (loss_GAN_XtoY + loss_GAN_YtoX) / 2

        # Cycle loss
        recov_X = cycle_gan.G_YtoX(fake_Y)
        loss_cycle_X = criterion_cycle(recov_X, real_X)

        recov_Y = cycle_gan.G_XtoY(fake_X)
        loss_cycle_Y = criterion_cycle(recov_Y, real_Y)

        loss_cycle = (loss_cycle_X + loss_cycle_Y) / 2

        # Total loss
        loss_G = loss_GAN + 10.0 * loss_cycle + 5.0 * loss_identity

        loss_G.backward()
        optimizer_G.step()

        # Adjust output range of generated images
        fake_Y = (fake_Y + 1) / 2  # Map from [-1, 1] to [0, 1]
        fake_X = (fake_X + 1) / 2  # Map from [-1, 1] to [0, 1]

        # ----------------------
        #  Train Discriminators
        # ----------------------

        # Discriminator X
        optimizer_D_X.zero_grad()

        loss_real_X = criterion_GAN(cycle_gan.D_X(real_X), valid)
        loss_fake_X = criterion_GAN(cycle_gan.D_X(fake_X.detach()), fake)

        loss_D_X = (loss_real_X + loss_fake_X) / 2

        loss_D_X.backward()
        optimizer_D_X.step()

        # Discriminator Y
        optimizer_D_Y.zero_grad()

        loss_real_Y = criterion_GAN(cycle_gan.D_Y(real_Y), valid)
        loss_fake_Y = criterion_GAN(cycle_gan.D_Y(fake_Y.detach()), fake)

        loss_D_Y = (loss_real_Y + loss_fake_Y) / 2

        loss_D_Y.backward()
        optimizer_D_Y.step()

        print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(older_dataloader)}] "
              f"[D loss: {loss_D_X.item() + loss_D_Y.item()}] [G loss: {loss_G.item()}]")


[Epoch 0/30] [Batch 0/66] [D loss: 0.5218207538127899] [G loss: 12.925529479980469]
[Epoch 0/30] [Batch 1/66] [D loss: 0.41605815291404724] [G loss: 11.643762588500977]
[Epoch 0/30] [Batch 2/66] [D loss: 0.4779323637485504] [G loss: 10.196147918701172]
[Epoch 0/30] [Batch 3/66] [D loss: 0.3108989894390106] [G loss: 8.92475700378418]
[Epoch 0/30] [Batch 4/66] [D loss: 0.26772312819957733] [G loss: 8.091504096984863]
[Epoch 0/30] [Batch 5/66] [D loss: 0.2384706437587738] [G loss: 7.534849643707275]
[Epoch 0/30] [Batch 6/66] [D loss: 0.1711549311876297] [G loss: 6.985177993774414]
[Epoch 0/30] [Batch 7/66] [D loss: 0.10584665834903717] [G loss: 6.555181503295898]
[Epoch 0/30] [Batch 8/66] [D loss: 0.07794133573770523] [G loss: 6.243775844573975]
[Epoch 0/30] [Batch 9/66] [D loss: 0.05199182219803333] [G loss: 5.860116004943848]
[Epoch 0/30] [Batch 10/66] [D loss: 0.03900352492928505] [G loss: 5.57840633392334]
[Epoch 0/30] [Batch 11/66] [D loss: 0.028393983840942383] [G loss: 5.4406719207

KeyboardInterrupt: 

## training한 모델 저장하기

In [ ]:
# Save the model
torch.save(cycle_gan.state_dict(), 'cycle_gan5.pth')

### trained된 모델로 이미지 넣어 결과물 보기

우선 저장된 모델 불러오기

In [59]:
import torch
from torchvision import transforms
from PIL import Image

# Assuming you have defined your CycleGAN model class (CycleGAN) and its architecture

# Instantiate the CycleGAN model
cycle_gan = CycleGAN()

# Load the trained model weights
model_path = '/content/cycle_gan4.pth'

cycle_gan.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
cycle_gan.eval()

# Move the model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cycle_gan.to(device)


CycleGAN(
  (G_XtoY): Generator(
    (main): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): ReLU(inplace=True)
      (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (13): ReLU(inplace=True)
      (14): ConvTrans

input 이미지 사진 transform하고 tensor로 변환하기

In [74]:
# Define transformations for the input image
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to the model's input size
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load and preprocess the test image
input_image_path = '/content/drive/MyDrive/deep_learning/97734_314000_4035 (1).jpg'
input_image = Image.open(input_image_path).convert('RGB')
input_tensor = transform(input_image).unsqueeze(0).to(device)


### output tensor 다시 이미지로 변환하고 결과물 저장하기

In [76]:
with torch.no_grad():
  # Perform translation
  output_tensor = cycle_gan.G_XtoY(input_tensor)  # Assuming you want to translate to domain Y

  # Convert the output tensor to a PIL image
  output_ts = output_tensor.squeeze().cuda()
  output_image = transforms.ToPILImage()(output_ts*0.5+0.5)

  # Display or save the translated image
  output_image.show()

  # Or save it to a file
  output_image.save('translated_image4.png')
